# Anomaly detection


In [1]:
from sqlplusapi import SqlPlus #/вычислить-выбросы
import pandas as pd
from utilites import read_excel,save_to_excel
from os import path,listdir,environ,mkdir
from functools import partial
import numpy as np
import scipy.stats as st
from sklearn.cluster import KMeans
from sklearn.svm import OneClassSVM
%matplotlib inline 
#from subprocess import call # MSK-M1-N5-2project.write_query()
#call('sqlplus -S cash/suffolk@ORAHCNV.NTDOMAIN.PRK @C:\\Reports\\anomalydetection\\day\\tempsql.sql >>C:\\Reports\\anomalydetection\\day\\1.csv', shell=True)

In [2]:
environ['NLS_LANG']='RUSSIAN_CIS.CL8MSWIN1251'
project_path=r'C:\Reports\anomalydetection\day'
project=SqlPlus(project_path,printlog=True) #projectdirectory,delimiter='|',sql_plus_options_new=None,printlog=False

INFO:root: Project initialisation (2017-02-20 11:25:45; sqlplusapi.py:37)


первым запросом вытащим чеки с ручным вводом (66)<br> вторым запрос вытащим чеки со скидкой

In [3]:
#sqlpath=r'C:\Scripts\SQL\anomaly.sql'

sqlpath2=r'C:\Scripts\SQL\anomaly_day.sql'
query_params={'BegPeriod':"'13.02.2017'",'EndPeriod':"'20.02.2017'"
              }

query_params_day={'BegPeriod':"'19.02.2017'",'EndPeriod':"'20.02.2017'"
                  #,'filter':"'and d.id_plu in (3270434,3270432,26764,1860,3289442,22248,2137723,3405362,3226024,3226021)'"
                  #'filter':''
              }
#query=project.set_sql_query(query_params,sqlpath)
query=project.set_sql_query(query_params_day,sqlpath2)
#project.printlines(query_2)#project.write_query(query=query)

Получаем Oracle id магазинов (из excel файла выбираем первый столбец и преобразуем в строку)

In [4]:
depart_codes_path='C:\Scripts\Dictionaries\departmentcodes.xlsx'
product_codes_path=r'C:\Scripts\Dictionaries\prod_short.xlsx'
read_excel(pd,depart_codes_path)
department_codes=list((read_excel(pd,depart_codes_path).ix[:,0]).apply(str))

conn_string=lambda department_name: "%s/%s@%s"% ("cash","suffolk","ORA%s.NTDOMAIN.PRK"%department_name)

### Загружаем себестоимость

In [ ]:
df_cost=pd.read_csv(r'C:\Reports\anomalydetection\day\cost.csv',sep=';',encoding='utf-8')
df_cost.columns=['ID_PLU','store','cost','qty']
df_cost['cost']=df_cost['cost'].apply(lambda x:np.float(x.strip().replace(',','.')))
df_cost['qty']=df_cost['qty'].apply(lambda x:np.float(x.strip().replace(',','.')))
df_cost['cost']=df_cost['cost']/df_cost['qty']
df_cost.drop('qty', axis=1, inplace=True)
df_cost.head(3)

In [ ]:
df_cost.head(3)

In [ ]:
df_cost[df_cost['ID_PLU']==57647]

## Запускаем асинхронные потоки для выгрузки данных

In [5]:
load_path=project.default_loaddirectory
load_path

'C:\\Reports\\anomalydetection\\day\\load'

In [6]:
%%time
project.prepareload(load_path)

#оборачиваем функцию выгрузки данных для распараллеливания
def worker(load_dir,sql_q,department_oraname):
    savepath_w=path.join(load_dir,'%s.csv'%department_oraname)
    project.FetchData(conn_string(department_oraname),savepath_w,sql_q)
    

worker=partial(worker,load_path,query)


project.ThreadPool(worker,department_codes)
#project.ThreadPool(worker_2,department_codes)

INFO:root: Created load directory C:\Reports\anomalydetection\day\load, content removed (2017-02-20 11:25:58; sqlplusapi.py:68)
INFO:root: Start ThreadPool: threads:96 async:True (2017-02-20 11:25:58; sqlplusapi.py:95)
INFO:root: Finished ThreadPool: threads:96 async:True (2017-02-20 11:30:54; sqlplusapi.py:107)


Wall time: 4min 56s


In [39]:
%%time
fetched_data_list=lambda savepath:list(map(lambda x:[path.splitext(x)[0],path.join(savepath,x)],
                           [f for f in listdir(savepath) if 'error_' not in f]))

df=pd.concat(list(map(project.read_loaded_csv,fetched_data_list(load_path))))

Wall time: 9.13 s


In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3149791 entries, 0 to 37436
Data columns (total 8 columns):
   ID_PAYDESK    int64
    ID_HEADER    object
       ID_PLU    int64
     TOTALSUM    object
          QTY    object
   CORRECTION    object
DATE_TIME        object
filename         object
dtypes: int64(2), object(6)
memory usage: 216.3+ MB


In [41]:
%%time
df.columns=list(map(lambda s:s.strip(),df.columns.values))
df['QTY']=df['QTY'].apply(lambda x:np.float(x.strip().replace(',','.')))
df['TOTALSUM']=df['TOTALSUM'].apply(lambda x:np.float(x.strip().replace(',','.')))
#df['PRICE']=df['PRICE'].apply(lambda x:np.float(x.strip().replace(',','.')))
df['CORRECTION']=df['CORRECTION'].apply(lambda x:np.float(x.strip().replace(',','.')))
#df["DATE_TIME"] = pd.to_datetime(df["DATE_TIME"])
#df['plu_sum']=df['PRICE']*df['QTY']
df['cor_sum']=df['CORRECTION']*df['QTY']
#df.drop_duplicates(inplace=True)

Wall time: 17.1 s


In [42]:
df.head(3)

,ID_PAYDESK,ID_HEADER,ID_PLU,TOTALSUM,QTY,CORRECTION,DATE_TIME,filename,cor_sum
0,1,1465416,3211788,95.0,1.0,0.00,19/02/2017 09:17:28,2X,0.00
1,16,508399,3244868,194.0,1.0,-0.03,19/02/2017 09:23:39,2X,-0.03
2,16,508400,3441471,12.9,1.0,0.00,19/02/2017 09:26:18,2X,0.00


In [43]:
hypermarkets=read_excel(pd,depart_codes_path)[['Код Oracle','Назв. Магазина','Завод']]
hypermarkets.columns=['filename','Назв. Магазина','store']
#df=pd.merge(df, hypermarkets[['Код Oracle','Назв. Магазина','Завод']], left_on='filename', right_on='Код Oracle')
hypermarkets.head(3)

,filename,Назв. Магазина,store
0,SM,Парк-Хаус,1001
1,VG,Волгоград-Сити,1002
2,HF,Фуражная,1004


In [44]:
plu_list=df[df['TOTALSUM']>10000] ['ID_PLU'].unique()
plu_list

array([3277894, 3276880, 2163677,   81514, 3166627, 3463294,    3068,
       2072272,     552, 3481619,   80185, 2089823,   17898, 2053835,
       3500209], dtype=int64)

### Алгоритмы Тьюки

http://ru.wikihow.com/%D0%B2%D1%8B%D1%87%D0%B8%D1%81%D0%BB%D0%B8%D1%82%D1%8C-%D0%B2%D1%8B%D0%B1%D1%80%D0%BE%D1%81%D1%8B
1. считаем интервартильный диапазон (по сути дисперсию, очищенную от выбросов)
2. считаем внутренними границами набора данных интерквартильный диапазон *1,5, внешними -//- *3 прибавленный к икв диапазону

Таким образом получаем примитивную классификацию.




## Заметки

scipy.stats.f_oneway  -The one-way ANOVA tests the null hypothesis that two or more groups have the same population mean. The test is applied to samples from two or more groups, possibly with differing sizes

# Дисперсионный анализ

http://easydan.com/arts/2016/anova/ 

Строго, классический однофакторный дисперсионный анализ применим, когда:
В отношении выборочных данных не отвергается гипотеза об их нормальности (для каждой выборки);
Выборки имеют равные теоретические дисперсии;

In [45]:
#критерий нормальности Шапиро Уилка //испключение трактовки p value(чем оно больше тем лучше)
#The Shapiro-Wilk test tests the null hypothesis that the data was drawn from a normal distribution.
def check_normality(series,alpha):
    if st.shapiro(series)[1] <= alpha:return False
    return True
#check_normality(st.norm.rvs(loc=5, scale=3, size=1000),0.05)
#st.mstats.kruskalwallis(st.norm.rvs(loc=5, scale=3, size=100))
#np.std([1,3,4,6], ddof=1)

In [46]:
plu_series=lambda data,plu_id:data[data['ID_PLU']==plu_id][['QTY','TOTALSUM']]

def plu_series_info(series_df,plu):
    series_ar=series_df[series_df.ix[:,0]>0]
    n=series_ar.shape[0]

    if n==0:return [plu,0,0,0,0,0,0,0,0,'error','no',0]
    if n<10 :return [plu,0,0,0,0,0,0,0,'not-enought data','no',0]
    series=series_ar.ix[:,0]
    min_plu=np.min(series)
    max_plu=np.max(series)
    median_plu=np.median(series)
    q1_plu=np.percentile(series, 25)
    q3_plu=np.percentile(series, 75)
    q_interval=q3_plu-q1_plu
    # std_plu=np.std(series, ddof=1)
    classif='normal'
    ip_q3_sum=0
    if max_plu>median_plu+3*q_interval:
        classif='outline'
        ip_q3_sum=np.sum(series_ar[series_ar.ix[:,0]>median_plu+3*q_interval].ix[:,1])
         #print(ip_q3_sum)
    elif max_plu>median_plu+1.5*q_interval:
        classif='smallanomaly'
        ip_q3_sum=np.sum(series_ar[series_ar.ix[:,0]>median_plu+2*q_interval].ix[:,1])
        
    return [plu,np.sum(series),min_plu,max_plu,median_plu,q1_plu,q3_plu,
            q_interval,classif,check_normality(series,0.05),ip_q3_sum]

In [47]:
df.head(3)

,ID_PAYDESK,ID_HEADER,ID_PLU,TOTALSUM,QTY,CORRECTION,DATE_TIME,filename,cor_sum
0,1,1465416,3211788,95.0,1.0,0.00,19/02/2017 09:17:28,2X,0.00
1,16,508399,3244868,194.0,1.0,-0.03,19/02/2017 09:23:39,2X,-0.03
2,16,508400,3441471,12.9,1.0,0.00,19/02/2017 09:26:18,2X,0.00


In [48]:
%%time
plu_series_info=pd.DataFrame([plu_series_info(plu_series(df,plu),plu) for plu in plu_list])
plu_series_info.columns=['plu','total_sum','min_plu','max_plu','median_plu','q1_plu','q3_plu',
                         'q_interval','classif','check_norm','q3_up_sum']
plu_series_info=plu_series_info.sort_values('q3_up_sum',ascending =False)

Wall time: 246 ms


C:\Users\ilya.Volchkov\AppData\Local\Continuum\Anaconda3\lib\site-packages\scipy\stats\morestats.py:1330: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


In [49]:
prd=read_excel(pd,product_codes_path)
prd.dropna(inplace=True)
prd['Материал']=prd['Материал'].apply(np.int)
plu_series_info=pd.merge(plu_series_info, prd, left_on='plu', right_on='Материал', how='left')
plu_series_info.drop('Материал', axis=1, inplace=True)
plu_series_info.head(3)

,plu,total_sum,min_plu,max_plu,median_plu,q1_plu,q3_plu,q_interval,classif,check_norm,q3_up_sum,name
0,552,7963.0,1.0,100.0,1.0,1.0,1.0,0.0,outline,False,507283.00,NESC.Кофе GOLD раст.ст/б 95г
1,81514,9926.0,1.0,216.0,1.0,1.0,2.0,1.0,outline,False,288376.76,Вода БОРЖОМИ минерал.ст/б 0.5л
2,2053835,799.0,1.0,30.0,1.0,1.0,2.0,1.0,outline,False,66293.00,Водка ЦАРСКАЯ ОРИГИНАЛЬНАЯ 40% 0.5л


## Кластеризация

In [50]:
plu_series_info=plu_series_info[plu_series_info['classif']=='outline']
plu_series_info.head(3)

,plu,total_sum,min_plu,max_plu,median_plu,q1_plu,q3_plu,q_interval,classif,check_norm,q3_up_sum,name
0,552,7963.0,1.0,100.0,1.0,1.0,1.0,0.0,outline,False,507283.00,NESC.Кофе GOLD раст.ст/б 95г
1,81514,9926.0,1.0,216.0,1.0,1.0,2.0,1.0,outline,False,288376.76,Вода БОРЖОМИ минерал.ст/б 0.5л
2,2053835,799.0,1.0,30.0,1.0,1.0,2.0,1.0,outline,False,66293.00,Водка ЦАРСКАЯ ОРИГИНАЛЬНАЯ 40% 0.5л


In [ ]:
df[df['ID_PLU']==57647]

In [51]:
%%time
plu_info_save=lambda plu:df[df['ID_PLU']==plu].sort_values('TOTALSUM',ascending =False)

knn_outliner_sum={}
class_1_min={}
class_2_min={}
df_plu_save=[]

rto_1={}
rto_2={}
qty_1={}
qty_2={}

cor_1={}
cor_2={}
fm_1={}
fm_2={}
for plu in plu_series_info['plu'].unique():
    df_plu=plu_info_save(plu)
    df_plu=pd.merge(df_plu, hypermarkets, on=['filename'],  how='left')
    #df_plu=pd.merge(df_plu, df_cost, on=['ID_PLU','store'],  how='left')
    #df_plu['FM']=df_plu['TOTALSUM']-df_plu['cost']*df_plu['QTY']
    #df_plu.drop('cost', axis=1, inplace=True)
    
    kmeans = KMeans(n_clusters=3)
    X=np.array(df_plu['QTY']).reshape(-1, 1)
    kmeans.fit(X)
    df_plu['plu']=plu
    df_plu['kmeans_cls']=kmeans.labels_
    
    uniq_labels= [el for el in df_plu['kmeans_cls'].unique()]
    uniq_labels_sorted=list(uniq_labels)
    uniq_labels_sorted.sort(reverse=True)
    labels_dict={uniq_labels[i]:uniq_labels_sorted[i] for i in range(len(uniq_labels))}
    df_plu['kmeans_cls']=df_plu['kmeans_cls'].apply(lambda x:labels_dict[x])
    
    knn_outliner_sum[plu]=np.sum(df_plu[df_plu['kmeans_cls']!=0]['TOTALSUM'])
    
    cls_min=[np.min(df_plu[df_plu['kmeans_cls']==i]['QTY'])  for i in range(3)]
    cls_min.sort()
    
    class_1_min[plu]=cls_min[0]
    class_2_min[plu]=cls_min[1]
    
    rto_1[plu]=np.sum(df_plu[df_plu['kmeans_cls']==1]['TOTALSUM']) 
    rto_2[plu]=np.sum(df_plu[df_plu['kmeans_cls']==2]['TOTALSUM']) 
    qty_1[plu]=np.sum(df_plu[df_plu['kmeans_cls']==1]['QTY']) 
    qty_2[plu]=np.sum(df_plu[df_plu['kmeans_cls']==2]['QTY'])   
    cor_1[plu]=np.sum(df_plu[df_plu['kmeans_cls']==1]['cor_sum']) 
    cor_2[plu]=np.sum(df_plu[df_plu['kmeans_cls']==2]['cor_sum']) 
    
    #fm_1[plu]=np.sum(df_plu[df_plu['kmeans_cls']==1]['FM']) 
    #fm_2[plu]=np.sum(df_plu[df_plu['kmeans_cls']==2]['FM'])    
    df_plu_save.append([plu,df_plu])

Wall time: 481 ms


In [52]:
plu_series_info['Knn_cluster_sum']=plu_series_info['plu'].apply(lambda x:knn_outliner_sum[x])
plu_series_info['sum_evaluate']= plu_series_info.loc[:, ['Knn_cluster_sum', 'q3_up_sum']].min(axis=1)
plu_series_info['class_1_min']=plu_series_info['plu'].apply(lambda x:class_1_min[x])
plu_series_info['class_2_min']=plu_series_info['plu'].apply(lambda x:class_2_min[x])

plu_series_info['rto_1']=plu_series_info['plu'].apply(lambda x:rto_1[x])
plu_series_info['rto_2']=plu_series_info['plu'].apply(lambda x:rto_2[x])
plu_series_info['qty_1']=plu_series_info['plu'].apply(lambda x:qty_1[x])
plu_series_info['qty_2']=plu_series_info['plu'].apply(lambda x:qty_2[x])
plu_series_info['cor_1']=plu_series_info['plu'].apply(lambda x:cor_1[x])
plu_series_info['cor_2']=plu_series_info['plu'].apply(lambda x:cor_2[x])

#plu_series_info['fm_1']=plu_series_info['plu'].apply(lambda x:fm_1[x])
#plu_series_info['fm_2']=plu_series_info['plu'].apply(lambda x:fm_2[x])

In [53]:
plu_series_info=plu_series_info.sort_values('sum_evaluate',ascending =False)
plu_series_info.reset_index(inplace=True,drop=True)
plu_series_info.head(3)

,plu,total_sum,min_plu,max_plu,median_plu,q1_plu,q3_plu,q_interval,classif,check_norm,...,Knn_cluster_sum,sum_evaluate,class_1_min,class_2_min,rto_1,rto_2,qty_1,qty_2,cor_1,cor_2
0,81514,9926.0,1.0,216.0,1.0,1.0,2.0,1.0,outline,False,...,253044.88,253044.88,0.0,8.0,229019.8,24025.08,3755.0,348.0,-70087.20,-419936.16
1,552,7963.0,1.0,100.0,1.0,1.0,1.0,0.0,outline,False,...,101738.00,101738.00,0.0,6.0,84838.0,16900.00,502.0,100.0,-9.46,-67.00
2,2053835,799.0,1.0,30.0,1.0,1.0,2.0,1.0,outline,False,...,66293.00,66293.00,0.0,5.0,47987.0,18306.00,133.0,54.0,-22499.94,0.00


In [54]:
concat_save_df=pd.concat([df[1] for df in df_plu_save])
concat_save_df=concat_save_df[concat_save_df['kmeans_cls']==1]
pt_hm=concat_save_df.pivot_table(values=['TOTALSUM','plu'],#columns=['Год','Квартал'],
                    index=['Назв. Магазина'],aggfunc={'TOTALSUM':np.sum,'plu':lambda x: len(x.unique())},fill_value=0 )
pt_hm=pt_hm.sort_values('TOTALSUM',ascending =False)
pt_hm.columns=['Сумма оптовых артикулов', 'к-во оптовых артикулов']

In [55]:
df_save=[['main',plu_series_info]]+df_plu_save
save_result_path=path.join(project_path,'anomaly11.xlsx')
writer = pd.ExcelWriter(save_result_path)
for el in df_save:
    el[1].to_excel(writer,str(el[0]),index =False)
pt_hm.to_excel(writer,'hm_pivot',index =True)
writer.save()

In [56]:
import win32com.client
Excel = win32com.client.Dispatch("Excel.Application")
#Excel.visible=True
wb = Excel.Workbooks.Open(save_result_path)
sh=wb.sheets(1)
for i,el in enumerate(plu_series_info['plu']):
    sh.Hyperlinks.Add (Anchor=sh.Cells(i+2, 1), Address="", SubAddress="'{}'!A1".format(str(el)))

n=sh.UsedRange.Rows.count+2
sh.cells(n,1).value='Нижняя оценка оптовых закупок по магазинам'
sh.cells(n,1).font.size=16
sh.cells(n,1).Font.Bold = True

ssh=wb.sheets('hm_pivot')
ssh.Range(ssh.Cells(1,1),ssh.Cells(pt_hm.shape[0],pt_hm.shape[1]+1)).Copy(sh.cells(n+1,1))

for i in range(wb.sheets.Count):
    wb.sheets(i+1).Columns.EntireColumn.AutoFit()
wb.Save()
wb.Close()
Excel.Quit()